## 初始化

### clone代码

In [ ]:
!git clone https://github.com/modelscope/modelscope-agent.git

### 安装特定依赖

In [ ]:
!cd modelscope-agent && pip install -r requirements.txt
! pip install transformers_stream_generator
! pip install invisible-watermark>=2.0

### 下载本地语料库

In [ ]:
!wget -P modelscope-agent/demo/ https://modelscope.oss-cn-beijing.aliyuncs.com/resource/agent/modelscope_qa.txt 

In [ ]:
import os
os.chdir('modelscope-agent/demo')


import sys
sys.path.append('../')

### 读取环境变量文件和工具config配置

In [ ]:
from dotenv import load_dotenv
import os
from modelscope.utils.config import Config

# 配置环境变量，里面需要配置modelscope token来remote调用AI模型
os.environ['TOOL_CONFIG_FILE'] ='../config/cfg_tool_template.json'
os.environ['OUTPUT_FILE_DIRECTORY'] = '/tmp'

# ModelScope Token从这里获取: https://modelscope.cn/my/myaccesstoken
print('请输入ModelScope Token，可以从这里获取: https://modelscope.cn/my/myaccesstoken')
os.environ['MODELSCOPE_API_TOKEN'] = input()

# 读取工具调用CONFIG文件，包括工具的名称和远程请求url
tool_cfg_file = os.getenv('TOOL_CONFIG_FILE')
print(tool_cfg_file)
tool_cfg = Config.from_file(tool_cfg_file)

### 中枢大模型启动

In [ ]:
from modelscope_agent.llm import LLMFactory
from modelscope_agent.agent import AgentExecutor


model_name = 'modelscope-agent-qwen-7b'
model_cfg = {
    'modelscope-agent-qwen-7b':{
        'type': 'modelscope',
        'model_id': 'damo/MSAgent-Qwen-7B',
        'model_revision': 'v1.0.2',
        'use_raw_generation_config': True,
        'custom_chat': True
    }
}


llm = LLMFactory.build_llm(model_name, model_cfg)



### 向量库配置

In [ ]:
from langchain.embeddings import ModelScopeEmbeddings, DashScopeEmbeddings
from langchain.vectorstores import FAISS
from modelscope_agent.retrieve import ToolRetrieval, KnowledgeRetrieval
import faiss

# 开源版本的向量库配置
model_id = 'damo/nlp_corom_sentence-embedding_chinese-base'
embeddings = ModelScopeEmbeddings(model_id=model_id)

# 基于阿里云DashScope向量Embedding的配置
#embeddings = DashScopeEmbeddings(model="text-embedding-v1")


file_path = 'modelscope_qa.txt'
knowledge_retrieval = KnowledgeRetrieval.from_file(file_path, embeddings, FAISS)

In [ ]:
agent = AgentExecutor(llm, tool_cfg, knowledge_retrieval=knowledge_retrieval)

In [ ]:
agent.run('pip install的时候有些包下载特别慢怎么办')